# Rusty Bargain Car Price Prediction

## Introduccion

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparacion de datos

In [50]:
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import time
import plotly.express as px
import lightgbm as lgb

In [51]:
df = pd.read_csv('/datasets/car_data.csv')
df.info()
df.describe()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Mileage                  0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

## Limpieza de datos

In [52]:
#Eliminacion de entradas con precio cero o extremadamente altos
df = df.query('Price > 100 and Price < 100000')

#Eliminacion de años improbables
df = df.query('RegistrationYear >= 1990 and RegistrationYear <= 2022')

#Relleno de valores faltantes
for column in ['VehicleType', 'FuelType', 'Gearbox', 'Model', 'NotRepaired']:
    df[column] = df[column].fillna('unknown')

#Eliminacion de columnas poco utiles
df = df.drop(columns=['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen'])

#Codificacion categorica
df = pd.get_dummies(df, drop_first=True)

#Separacion de variables
target = df['Price']
features = df.drop(columns=['Price'])

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

## Definicion de metrica RECM

In [32]:
def recm(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

## Regresion lineal (para hacer la prueba de cordura)

In [33]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
pred_lr = model_lr.predict(X_test)
print("RECM (Linear Regression):", recm(y_test, pred_lr))

RECM (Linear Regression): 2559.373659775109


## Arbol de decision y bosque aleatorio

In [36]:
#Arbol
model_tree = DecisionTreeRegressor(max_depth=10, random_state=42)
model_tree.fit(X_train, y_train)
pred_tree = model_tree.predict(X_test)
print("RECM (Decision Tree):", recm(y_test, pred_tree))

RECM (Decision Tree): 1956.9194011205634


In [35]:
#Bosque Aleatorio
model_rf = RandomForestRegressor(n_estimators=30, max_depth=15,n_jobs=1, random_state=42)
model_rf.fit(X_train, y_train)
pred_rf = model_rf.predict(X_test)
print("RECM (Random Forest):", recm(y_test, pred_rf))

RECM (Random Forest): 1656.3337569709167


## LightGBM

In [37]:
model_lgbm = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.1, n_estimators=30)
model_lgbm.fit(X_train, y_train)
pred_lgbm = model_lgbm.predict(X_test)
print("RECM (LightGBM):", recm(y_test, pred_lgbm))

RECM (LightGBM): 1881.0463170238854


## CatBoost

In [38]:
model_cat = CatBoostRegressor(verbose=0, iterations=100, learning_rate=0.1)
model_cat.fit(X_train, y_train)
pred_cat = model_cat.predict(X_test)
print("RECM (CatBoost):", recm(y_test, pred_cat))

RECM (CatBoost): 1791.8967313104981


## XGBoost

In [39]:
model_xgb = XGBRegressor(n_estimators=30, learning_rate=0.1, max_depth=6)
model_xgb.fit(X_train, y_train)
pred_xgb = model_xgb.predict(X_test)
print("RECM (XGBoost):", recm(y_test, pred_xgb))

RECM (XGBoost): 1871.6419919348762


## Comparacion de rendimiento

In [40]:
def measure_model(model, name):
    start = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start

    start = time.time()
    preds = model.predict(X_test)
    predict_time = time.time() - start

    score = recm(y_test, preds)
    print(f"{name}: RECM={score:.2f}, Train Time={train_time:.2f}s, Predict Time={predict_time:.2f}s")

measure_model(LinearRegression(), "Regresion Lineal")
measure_model(RandomForestRegressor(n_estimators=30), "Bosque Aleatorio")

Regresion Lineal: RECM=2559.37, Train Time=8.53s, Predict Time=0.18s
Bosque Aleatorio: RECM=1550.23, Train Time=107.53s, Predict Time=0.99s


## Hallazgos

Desde mi punto de vista y viendo los resultados el mejor modelo en precision fue Random Forest, seguido de CatBoost y Arbol de decision.

Regresion lineal tuvo el peor desempeño como se esperaba y me sirvio bien como prueba de cordura ya que cualquier modelo con peor RECM que el lineal tendria que ser reexaminado.

Ahora viendo los tiempos de entrenamiento y prediccion me di cuenta que aunque el bosque aleatorio fue el mas preciso, entrenarlo fue el mas costoso en tiempo.

## Conclusion

En base a los resultados obtenidos, lorge determinar que el modelo de Bosque Aleatorio logra la mejor calidad de prediccion de acuerdo a la metrica RECM.

Por otro lado CatBoost y LightGBM muestran un desempeño competitivo en precision, con tiempos de entrenamiento moderados y ventajas especificas como el manejo automatico de variables categoricas.

Finalmente, la Regresion Lineal cumple su funcion como prueba de cordura, ya que al tener un RECM significativamente mas alto, valida que los modelos mas sofisticados estan mejor generalizando.